<a href="https://colab.research.google.com/github/joanizba/Spotifypred/blob/dev_joan/Regresiones_L_y_P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
file_path ='/content/drive/MyDrive/PROYECTO_FINAL/dataset/playlist_2010to2022.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Regresión lineal**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import warnings

# Ignorar advertencias para mantener la salida limpia (opcional)
warnings.filterwarnings('ignore')

# --- 1. Carga de Datos ---
# Asegúrate de que tu archivo CSV esté en tu Google Drive o súbelo a Colab
# Si está en Drive, necesitas montar tu Drive primero:
# from google.colab import drive
# drive.mount('/content/drive')
# file_path = '/content/drive/My Drive/tu_carpeta/tu_dataset.csv' # Cambia esto a la ruta correcta


data = pd.read_csv(file_path)
df = data.copy()
print(f"Datos cargados exitosamente desde '{file_path}'")
print(f"Número de filas: {df.shape[0]}, Número de columnas: {df.shape[1]}")

# --- 2. Selección de Características y Preprocesamiento ---

# Seleccionar características numéricas relevantes para la predicción
# Excluimos IDs, nombres, URLs y géneros (que requerirían un preprocesamiento más complejo)
# También excluimos el target 'track_popularity' de las features X
numeric_features = [
    'year', 'track_popularity', 'artist_popularity', 'danceability',
    'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
    'time_signature'
]

# Asegurarnos de que solo trabajamos con columnas que existen y son numéricas
existing_features = [col for col in numeric_features if col in df.columns]
df_subset = df[existing_features].copy()

# Convertir columnas a numéricas si es necesario (forzando errores a NaN)
for col in existing_features:
    df_subset[col] = pd.to_numeric(df_subset[col], errors='coerce')

# Manejar valores faltantes (una estrategia simple: eliminar filas con NaN en las columnas seleccionadas)
print(f"\nValores faltantes antes de limpiar:\n{df_subset.isnull().sum()}")
df_subset.dropna(inplace=True)
print(f"\nForma del dataset después de limpiar NaNs: {df_subset.shape}")

if df_subset.empty:
    print("Error: El dataset quedó vacío después de eliminar filas con valores faltantes.")
    exit()

# Separar características (X) y variable objetivo (y)
X = df_subset.drop('track_popularity', axis=1)
y = df_subset['track_popularity']

# Verificar que aún tenemos datos
if X.empty or y.empty:
    print("Error: No quedan datos suficientes después de la selección de características y limpieza.")
    exit()

print("\nCaracterísticas seleccionadas (X):")
print(X.columns.tolist())
print("\nVariable objetivo (y): track_popularity")

# --- 3. División en Conjuntos de Entrenamiento y Prueba ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\nTamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]} muestras")

# --- 4. Normalización (Escalado) de Datos ---
# Es crucial escalar DESPUÉS de dividir para evitar fuga de datos del test set al scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nDatos normalizados (primeras 5 filas de entrenamiento):")
print(X_train_scaled[:5])

# --- 5. Función para Encontrar el Mejor Alpha ---
def find_best_alpha(model_type, alphas, X_train, y_train, X_test, y_test):
    """
    Entrena modelos Ridge o Lasso con diferentes alphas y encuentra el mejor.

    Args:
        model_type (str): 'ridge' o 'lasso'.
        alphas (list): Lista de valores alpha a probar.
        X_train, y_train: Datos de entrenamiento.
        X_test, y_test: Datos de prueba.

    Returns:
        tuple: (mejor_alpha, mejor_r2, mejor_mse, mejor_modelo)
    """
    best_r2 = -np.inf  # Inicializar con un valor muy bajo para R^2
    best_mse = np.inf   # Inicializar con un valor muy alto para MSE
    best_alpha = None
    best_model = None

    print(f"\n--- Buscando el mejor alpha para {model_type.capitalize()} ---")
    print(f"Alphas a probar: {alphas}")

    for alpha in alphas:
        if model_type == 'ridge':
            model = Ridge(alpha=alpha, random_state=42)
        elif model_type == 'lasso':
            # Aumentar max_iter puede ser necesario para que Lasso converja
            model = Lasso(alpha=alpha, random_state=42, max_iter=10000)
        else:
            raise ValueError("model_type debe ser 'ridge' o 'lasso'")

        # Entrenar el modelo
        model.fit(X_train, y_train)

        # Predecir en el conjunto de prueba
        y_pred = model.predict(X_test)

        # Evaluar el modelo
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"  Alpha={alpha:.4f} -> R² = {r2:.4f}, MSE = {mse:.4f}")

        # Actualizar si encontramos un mejor modelo (basado en R²)
        if r2 > best_r2:
            best_r2 = r2
            best_mse = mse
            best_alpha = alpha
            best_model = model

    print(f"\nMejor alpha para {model_type.capitalize()}: {best_alpha}")
    print(f"  Mejor R²: {best_r2:.4f}")
    print(f"  Mejor MSE: {best_mse:.4f}")

    return best_alpha, best_r2, best_mse, best_model

# --- 6. Definir Rangos de Alpha y Ejecutar la Búsqueda ---

# Rangos de alpha especificados
ridge_alphas = list(range(2, 11)) # Enteros del 2 al 10
lasso_alphas = [0.1, 1, 10]

# Encontrar el mejor alpha para Ridge
best_alpha_ridge, best_r2_ridge, best_mse_ridge, best_ridge_model = find_best_alpha(
    'ridge', ridge_alphas, X_train_scaled, y_train, X_test_scaled, y_test
)

# Encontrar el mejor alpha para Lasso
best_alpha_lasso, best_r2_lasso, best_mse_lasso, best_lasso_model = find_best_alpha(
    'lasso', lasso_alphas, X_train_scaled, y_train, X_test_scaled, y_test
)

# --- 7. Comparación y Resultados Finales ---

print("\n--- Resumen de los Mejores Modelos Regularizados ---")
print(f"Mejor Ridge: Alpha={best_alpha_ridge}, R²={best_r2_ridge:.4f}, MSE={best_mse_ridge:.4f}")
print(f"Mejor Lasso: Alpha={best_alpha_lasso}, R²={best_r2_lasso:.4f}, MSE={best_mse_lasso:.4f}")

# (Opcional) Entrenar un modelo de Regresión Lineal simple como baseline
print("\n--- Evaluando Regresión Lineal Simple (Baseline) ---")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
print(f"Regresión Lineal Simple: R²={r2_lr:.4f}, MSE={mse_lr:.4f}")

# (Opcional) Ver los coeficientes del mejor modelo Lasso
if best_lasso_model:
    print(f"\n--- Coeficientes del Mejor Modelo Lasso (Alpha={best_alpha_lasso}) ---")
    lasso_coeffs = pd.DataFrame({
        'Característica': X.columns,
        'Coeficiente': best_lasso_model.coef_
    })
    # Mostrar coeficientes no nulos (Lasso realiza selección de características)
    print(lasso_coeffs[lasso_coeffs['Coeficiente'] != 0].sort_values(by='Coeficiente', ascending=False))

# (Opcional) Ver los coeficientes del mejor modelo Ridge
if best_ridge_model:
     print(f"\n--- Coeficientes del Mejor Modelo Ridge (Alpha={best_alpha_ridge}) ---")
     ridge_coeffs = pd.DataFrame({
        'Característica': X.columns,
        'Coeficiente': best_ridge_model.coef_
     })
     print(ridge_coeffs.sort_values(by='Coeficiente', ascending=False))

print("\n¡Análisis completado!")

Datos cargados exitosamente desde '/content/drive/MyDrive/PROYECTO_FINAL/dataset/playlist_2010to2022.csv'
Número de filas: 2300, Número de columnas: 23

Valores faltantes antes de limpiar:
year                 0
track_popularity     0
artist_popularity    0
danceability         1
energy               1
key                  1
loudness             1
mode                 1
speechiness          1
acousticness         1
instrumentalness     1
liveness             1
valence              1
tempo                1
duration_ms          1
time_signature       1
dtype: int64

Forma del dataset después de limpiar NaNs: (2299, 16)

Características seleccionadas (X):
['year', 'artist_popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

Variable objetivo (y): track_popularity

Tamaño del conjunto de entrenamiento: 1839 muestras
Tamaño del conjunto de prueba: 460 muestras

D

In [ ]:
import pandas as pd
import numpy as np
import ast # Para convertir string de listas a listas reales
from collections import Counter
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, KBinsDiscretizer # Para binning y OHE
import warnings

# Ignorar advertencias (opcional)
warnings.filterwarnings('ignore')

# --- Carga y Limpieza Inicial (Asegúrate de que esto se ejecute primero) ---
# (Copio las partes relevantes del código anterior para que sea autocontenido)


data = pd.read_csv(file_path)
df = data.copy()
print(f"Datos cargados exitosamente desde '{file_path}'")
print(f"Número de filas: {df.shape[0]}, Número de columnas: {df.shape[1]}")
# Seleccionar columnas relevantes iniciales (incluyendo genres ahora)
initial_features = [
    'track_popularity', 'year', 'artist_popularity', 'danceability',
    'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
    'time_signature', 'artist_genres' # Incluimos géneros
]

existing_features = [col for col in initial_features if col in df.columns]
df_subset = df[existing_features].copy()

# Convertir columnas numéricas (excluyendo genres por ahora)
numeric_cols_to_check = [col for col in existing_features if col != 'artist_genres' and col != 'track_popularity']
for col in numeric_cols_to_check:
    df_subset[col] = pd.to_numeric(df_subset[col], errors='coerce')

# Manejar valores faltantes en columnas numéricas y target (eliminar filas)
cols_to_check_na = [col for col in existing_features if col != 'artist_genres']
print(f"\nValores faltantes antes de limpiar (en subset inicial):\n{df_subset[cols_to_check_na].isnull().sum()}")
df_subset.dropna(subset=cols_to_check_na, inplace=True)
print(f"\nForma del dataset después de limpiar NaNs numéricos: {df_subset.shape}")

# Asegurarse de que 'artist_genres' sea string y manejar NaNs (reemplazar con '[]')
if 'artist_genres' in df_subset.columns:
    df_subset['artist_genres'] = df_subset['artist_genres'].fillna('[]').astype(str)
else:
    print("Advertencia: La columna 'artist_genres' no se encontró.")

if df_subset.empty:
    print("Error: El dataset quedó vacío después de la limpieza inicial.")
    exit()

# Separar X e y TEMPORALMENTE para aplicar ingeniería solo a X
X_temp = df_subset.drop('track_popularity', axis=1)
y = df_subset['track_popularity'] # y ya está limpio y listo

print("\n--- Iniciando Ingeniería de Características ---")

# Crear una copia para no modificar X_temp directamente en cada paso
X_engineered = X_temp.copy()

# --- 1. Procesamiento de Géneros (artist_genres) ---
if 'artist_genres' in X_engineered.columns:
    print("\nProcesando 'artist_genres'...")
    # Función segura para convertir string de lista a lista
    def parse_genre_list(genres_str):
        try:
            # Evaluar la cadena para obtener la lista
            genres = ast.literal_eval(genres_str)
            # Asegurarse de que sea una lista (puede ser None o algo más si la celda estaba vacía o mal formateada)
            if isinstance(genres, list):
                return genres
            else:
                return [] # Devolver lista vacía si no es una lista
        except (ValueError, SyntaxError):
            # Si falla la evaluación (ej. no es formato lista), devolver lista vacía
            return []

    # Aplicar la función de parseo
    X_engineered['genre_list'] = X_engineered['artist_genres'].apply(parse_genre_list)

    # Contar la frecuencia de cada género individual
    all_genres = [genre for sublist in X_engineered['genre_list'] for genre in sublist]
    genre_counts = Counter(all_genres)

    # Definir cuántos géneros top queremos codificar (ajusta N según necesites)
    N_TOP_GENRES = 20
    top_genres = [genre for genre, count in genre_counts.most_common(N_TOP_GENRES)]
    print(f"Top {N_TOP_GENRES} géneros encontrados: {top_genres}")

    # Crear columnas One-Hot para los géneros top
    for genre in top_genres:
        # Nueva columna se llamará 'genre_...'
        col_name = f'genre_{genre.replace(" ", "_").replace("-", "_")}' # Limpiar nombre
        X_engineered[col_name] = X_engineered['genre_list'].apply(lambda x: 1 if genre in x else 0)

    # Eliminar columnas intermedias y originales de género
    X_engineered.drop(['artist_genres', 'genre_list'], axis=1, inplace=True)
    print(f"Se añadieron {N_TOP_GENRES} columnas de género (One-Hot Encoded).")
else:
    print("\n'artist_genres' no presente, saltando procesamiento de géneros.")


# --- 2. Transformaciones No Lineales ---
print("\nAplicando transformaciones no lineales...")
# Logaritmo (usar log1p para manejar ceros) a características potencialmente sesgadas
# Asegúrate de que estas columnas existan antes de intentar transformarlas
log_features = ['duration_ms', 'speechiness', 'liveness', 'instrumentalness']
for col in log_features:
    if col in X_engineered.columns:
        # Verificar si hay valores negativos antes de aplicar log
        if (X_engineered[col] < 0).any():
            print(f"Advertencia: La columna '{col}' contiene valores negativos. No se aplicará log.")
        else:
            X_engineered[f'{col}_log'] = np.log1p(X_engineered[col])
            print(f" - Columna '{col}_log' creada.")
            # Decisión: ¿Eliminar la original? Por ahora la dejamos, Lasso podría eliminarla si no es útil.
            # X_engineered.drop(col, axis=1, inplace=True)


# Términos cuadráticos para algunas características clave
poly_features = ['artist_popularity', 'danceability', 'loudness', 'energy', 'valence']
for col in poly_features:
     if col in X_engineered.columns:
        X_engineered[f'{col}_sq'] = X_engineered[col]**2
        print(f" - Columna '{col}_sq' creada.")
        # Nuevamente, dejamos la original por ahora


# --- 3. Creación de Interacciones ---
print("\nCreando características de interacción...")
interactions = [
    ('danceability', 'energy'),
    ('valence', 'energy'),
    #('artist_popularity', 'acousticness') # Comentado, puedes añadir las que creas relevantes
]

for col1, col2 in interactions:
    if col1 in X_engineered.columns and col2 in X_engineered.columns:
        X_engineered[f'{col1}_x_{col2}'] = X_engineered[col1] * X_engineered[col2]
        print(f" - Interacción '{col1}_x_{col2}' creada.")

# Podrías crear interacciones con los géneros OHE si crees que es relevante
# Ejemplo: interactuar danceability con el género pop
# if 'genre_pop' in X_engineered.columns and 'danceability' in X_engineered.columns:
#    X_engineered['danceability_x_pop'] = X_engineered['danceability'] * X_engineered['genre_pop']
#    print(" - Interacción 'danceability_x_pop' creada.")


# --- 4. Binning (Agrupación) ---
print("\nAplicando binning a 'artist_popularity'...")
if 'artist_popularity' in X_engineered.columns:
    N_BINS = 5 # Número de contenedores (ej. 5 para quintiles)
    bin_col_name = 'artist_popularity_binned'

    # Usaremos KBinsDiscretizer para manejar mejor los bordes y codificación
    # Estrategia 'quantile' para tener aprox. el mismo número de muestras por bin
    try:
        discretizer = KBinsDiscretizer(n_bins=N_BINS, encode='ordinal', strategy='quantile')
        # Usar reshape(-1, 1) ya que espera una entrada 2D
        X_engineered[bin_col_name] = discretizer.fit_transform(X_engineered[['artist_popularity']])
        print(f" - Columna '{bin_col_name}' creada con {N_BINS} bins (ordinal).")

        # Ahora, aplicar One-Hot Encoding a los bins, ya que son categóricos
        ohe_binned = pd.get_dummies(X_engineered[bin_col_name], prefix=bin_col_name)
        # Concatenar las nuevas columnas OHE
        X_engineered = pd.concat([X_engineered, ohe_binned], axis=1)
        # Eliminar la columna ordinal intermedia
        X_engineered.drop(bin_col_name, axis=1, inplace=True)
        print(f" - Columnas One-Hot creadas para '{bin_col_name}'.")

        # Decisión: ¿Eliminar la 'artist_popularity' original ahora que está binarizada?
        # Es común hacerlo para evitar redundancia con los bins.
        X_engineered.drop('artist_popularity', axis=1, inplace=True)
        print(f" - Columna 'artist_popularity' original eliminada.")

    except Exception as e:
        print(f"Error durante el binning de 'artist_popularity': {e}. Saltando este paso.")

else:
    print("'artist_popularity' no encontrada, saltando binning.")


# --- Finalización ---
print("\n--- Ingeniería de Características Completada ---")

# Eliminar columnas que puedan contener NaNs introducidos (si los hubiera)
# X_engineered.dropna(inplace=True) # Cuidado: esto podría eliminar muchas filas si alguna transformación falló

# Asegurarse de que todas las columnas sean numéricas
# (OHE y transformaciones deberían serlo, pero es bueno verificar)
non_numeric_cols = X_engineered.select_dtypes(exclude=np.number).columns
if len(non_numeric_cols) > 0:
    print(f"\nAdvertencia: Se encontraron columnas no numéricas después de la ingeniería: {list(non_numeric_cols)}")
    print("Intentando convertir a numérico o eliminándolas...")
    # Intenta convertir forzadamente, si falla, considera eliminarlas o revisa el paso que las creó
    for col in non_numeric_cols:
        X_engineered[col] = pd.to_numeric(X_engineered[col], errors='coerce')
    X_engineered.dropna(axis=1, inplace=True) # Elimina columnas que no se pudieron convertir

# Verificar si todavía quedan NaNs en las filas
if X_engineered.isnull().sum().sum() > 0:
     print("\nAdvertencia: Aún quedan valores NaN después de la ingeniería. Considera imputar o eliminar filas.")
     # Opción: eliminar filas con NaN restantes
     # original_rows = X_engineered.shape[0]
     # y = y[X_engineered.index] # Asegurar alineación antes de dropear filas en X
     # X_engineered.dropna(axis=0, inplace=True)
     # print(f"Se eliminaron {original_rows - X_engineered.shape[0]} filas con NaN.")


print(f"\nForma final de X (características ingenierizadas): {X_engineered.shape}")
print("\nColumnas finales en X_engineered:")
print(X_engineered.columns.tolist())
print("\nPrimeras filas de X_engineered:")
print(X_engineered.head())

# --- ¡Listo para Modelar! ---
# Ahora puedes usar X_engineered y 'y' para dividir, escalar y entrenar tus modelos
# como en el código anterior.

# Ejemplo de cómo continuar:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

# 1. División Train/Test
X_train, X_test, y_train, y_test = train_test_split(X_engineered, y, test_size=0.2, random_state=42)

# 2. Escalado (IMPORTANTE: Escalar DESPUÉS de dividir y DESPUÉS de la ingeniería)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Búsqueda de Alpha y Entrenamiento (usando la función find_best_alpha definida antes)
best_alpha_ridge, best_r2_ridge, best_mse_ridge, best_ridge_model = find_best_alpha('ridge', ridge_alphas, X_train_scaled, y_train, X_test_scaled, y_test )
best_alpha_lasso, best_r2_lasso, best_mse_lasso, best_lasso_model = find_best_alpha('lasso', lasso_alphas, X_train_scaled, y_train, X_test_scaled, y_test)

# 4. Evaluación y Comparación
# ... (código de evaluación anterior) ...
print("\n--- Resumen de los Mejores Modelos Regularizados ---")
print(f"Mejor Ridge: Alpha={best_alpha_ridge}, R²={best_r2_ridge:.4f}, MSE={best_mse_ridge:.4f}")
print(f"Mejor Lasso: Alpha={best_alpha_lasso}, R²={best_r2_lasso:.4f}, MSE={best_mse_lasso:.4f}")


Datos cargados exitosamente desde '/content/drive/MyDrive/PROYECTO_FINAL/dataset/playlist_2010to2022.csv'
Número de filas: 2300, Número de columnas: 23

Valores faltantes antes de limpiar (en subset inicial):
track_popularity     0
year                 0
artist_popularity    0
danceability         1
energy               1
key                  1
loudness             1
mode                 1
speechiness          1
acousticness         1
instrumentalness     1
liveness             1
valence              1
tempo                1
duration_ms          1
time_signature       1
dtype: int64

Forma del dataset después de limpiar NaNs numéricos: (2299, 17)

--- Iniciando Ingeniería de Características ---

Procesando 'artist_genres'...
Top 20 géneros encontrados: ['pop', 'dance pop', 'rap', 'pop rap', 'hip hop', 'r&b', 'urban contemporary', 'trap', 'southern hip hop', 'modern rock', 'rock', 'canadian pop', 'edm', 'hip pop', 'pop dance', 'atl hip hop', 'uk pop', 'neo mellow', 'gangster rap', 'post

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler # Para escalar características
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Cargar los Datos ---
# !!! Reemplaza 'tu_archivo.csv' con el nombre real de tu archivo de datos !!!

data = pd.read_csv(file_path)
df
print(f"Datos cargados exitosamente desde '{file_path}'")
print(f"Número de filas: {df.shape[0]}, Número de columnas: {df.shape[1]}")
# print("\nInformación del DataFrame:")
# df.info()


# --- 2. Selección de Características y Preprocesamiento ---

# Seleccionar características numéricas relevantes para la predicción
# Excluimos IDs, nombres, URLs y géneros (que requerirían un preprocesamiento más complejo)
# También excluimos el target 'track_popularity' de las features X
numeric_features = [
    'year', 'track_popularity', 'artist_popularity', 'danceability',
    'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
    'time_signature'
]

# Asegurarnos de que solo trabajamos con columnas que existen y son numéricas
existing_features = [col for col in numeric_features if col in df.columns]
df_subset = df[existing_features].copy()

# Convertir columnas a numéricas si es necesario (forzando errores a NaN)
for col in existing_features:
    df_subset[col] = pd.to_numeric(df_subset[col], errors='coerce')

# Manejar valores faltantes (una estrategia simple: eliminar filas con NaN en las columnas seleccionadas)
print(f"\nValores faltantes antes de limpiar:\n{df_subset.isnull().sum()}")
df_subset.dropna(inplace=True)
print(f"\nForma del dataset después de limpiar NaNs: {df_subset.shape}")

if df_subset.empty:
    print("Error: El dataset quedó vacío después de eliminar filas con valores faltantes.")
    exit()

# Separar características (X) y variable objetivo (y)
X = df_subset.drop('track_popularity', axis=1)
y = df_subset['track_popularity']

# Verificar que aún tenemos datos
if X.empty or y.empty:
    print("Error: No quedan datos suficientes después de la selección de características y limpieza.")
    exit()

print("\nCaracterísticas seleccionadas (X):")
print(X.columns.tolist())
print("\nVariable objetivo (y): track_popularity")

# --- 3. División en Conjuntos de Entrenamiento y Prueba ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\nTamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]} muestras")

# --- 4. Normalización (Escalado) de Datos ---
# Es crucial escalar DESPUÉS de dividir para evitar fuga de datos del test set al scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nDatos normalizados (primeras 5 filas de entrenamiento):")
print(X_train_scaled[:5])

# --- 5. Función para Encontrar el Mejor Alpha ---
def find_best_alpha(model_type, alphas, X_train, y_train, X_test, y_test):
    """
    Entrena modelos Ridge o Lasso con diferentes alphas y encuentra el mejor.

    Args:
        model_type (str): 'ridge' o 'lasso'.
        alphas (list): Lista de valores alpha a probar.
        X_train, y_train: Datos de entrenamiento.
        X_test, y_test: Datos de prueba.

    Returns:
        tuple: (mejor_alpha, mejor_r2, mejor_mse, mejor_modelo)
    """
    best_r2 = -np.inf  # Inicializar con un valor muy bajo para R^2
    best_mse = np.inf   # Inicializar con un valor muy alto para MSE
    best_alpha = None
    best_model = None

    print(f"\n--- Buscando el mejor alpha para {model_type.capitalize()} ---")
    print(f"Alphas a probar: {alphas}")

    for alpha in alphas:
        if model_type == 'ridge':
            model = Ridge(alpha=alpha, random_state=42)
        elif model_type == 'lasso':
            # Aumentar max_iter puede ser necesario para que Lasso converja
            model = Lasso(alpha=alpha, random_state=42, max_iter=10000)
        else:
            raise ValueError("model_type debe ser 'ridge' o 'lasso'")

        # Entrenar el modelo
        model.fit(X_train, y_train)

        # Predecir en el conjunto de prueba
        y_pred = model.predict(X_test)

        # Evaluar el modelo
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"  Alpha={alpha:.4f} -> R² = {r2:.4f}, MSE = {mse:.4f}")

        # Actualizar si encontramos un mejor modelo (basado en R²)
        if r2 > best_r2:
            best_r2 = r2
            best_mse = mse
            best_alpha = alpha
            best_model = model

    print(f"\nMejor alpha para {model_type.capitalize()}: {best_alpha}")
    print(f"  Mejor R²: {best_r2:.4f}")
    print(f"  Mejor MSE: {best_mse:.4f}")

    return best_alpha, best_r2, best_mse, best_model

# --- 6. Definir Rangos de Alpha y Ejecutar la Búsqueda ---

# Rangos de alpha especificados
ridge_alphas = list(range(2, 11)) # Enteros del 2 al 10
lasso_alphas = [0.1, 1, 10]

# Encontrar el mejor alpha para Ridge
best_alpha_ridge, best_r2_ridge, best_mse_ridge, best_ridge_model = find_best_alpha(
    'ridge', ridge_alphas, X_train_scaled, y_train, X_test_scaled, y_test
)

# Encontrar el mejor alpha para Lasso
best_alpha_lasso, best_r2_lasso, best_mse_lasso, best_lasso_model = find_best_alpha(
    'lasso', lasso_alphas, X_train_scaled, y_train, X_test_scaled, y_test
)

# --- 7. Comparación y Resultados Finales ---

print("\n--- Resumen de los Mejores Modelos Regularizados ---")
print(f"Mejor Ridge: Alpha={best_alpha_ridge}, R²={best_r2_ridge:.4f}, MSE={best_mse_ridge:.4f}")
print(f"Mejor Lasso: Alpha={best_alpha_lasso}, R²={best_r2_lasso:.4f}, MSE={best_mse_lasso:.4f}")

# (Opcional) Entrenar un modelo de Regresión Lineal simple como baseline
print("\n--- Evaluando Regresión Lineal Simple (Baseline) ---")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
print(f"Regresión Lineal Simple: R²={r2_lr:.4f}, MSE={mse_lr:.4f}")

# (Opcional) Ver los coeficientes del mejor modelo Lasso
if best_lasso_model:
    print(f"\n--- Coeficientes del Mejor Modelo Lasso (Alpha={best_alpha_lasso}) ---")
    lasso_coeffs = pd.DataFrame({
        'Característica': X.columns,
        'Coeficiente': best_lasso_model.coef_
    })
    # Mostrar coeficientes no nulos (Lasso realiza selección de características)
    print(lasso_coeffs[lasso_coeffs['Coeficiente'] != 0].sort_values(by='Coeficiente', ascending=False))

# (Opcional) Ver los coeficientes del mejor modelo Ridge
if best_ridge_model:
     print(f"\n--- Coeficientes del Mejor Modelo Ridge (Alpha={best_alpha_ridge}) ---")
     ridge_coeffs = pd.DataFrame({
        'Característica': X.columns,
        'Coeficiente': best_ridge_model.coef_
     })
     print(ridge_coeffs.sort_values(by='Coeficiente', ascending=False))

print("\n¡Análisis completado!")

Error: No se encontró el archivo en la ruta: tu_dataset.csv
Por favor, asegúrate de que el archivo exista y la ruta sea correcta.

Valores faltantes antes de limpiar:
year                 0
track_popularity     0
artist_popularity    0
danceability         1
energy               1
key                  1
loudness             1
mode                 1
speechiness          1
acousticness         1
instrumentalness     1
liveness             1
valence              1
tempo                1
duration_ms          1
time_signature       1
dtype: int64

Forma del dataset después de limpiar NaNs: (2299, 16)

Características seleccionadas (X):
['year', 'artist_popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

Variable objetivo (y): track_popularity

Tamaño del conjunto de entrenamiento: 1839 muestras
Tamaño del conjunto de prueba: 460 muestras

Datos normalizados (pri

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.metrics import mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore', category=FutureWarning) # Ignorar warnings futuros de scikit-learn

# --- 1. Cargar y Preparar los Datos ---


df = pd.read_csv(file_path)

print("Dataset cargado. Primeras filas:")
print(df.head())
print("\nInformación del Dataset:")
df.info()

# --- Selección de Características y Objetivo ---
# Columnas potencialmente irrelevantes o identificadores
cols_to_drop = ['playlist_url', 'track_id', 'track_name', 'album', 'artist_id', 'artist_name', 'artist_genres']
# Nota: 'artist_genres' es complejo. Podría requerir NLP o dividir/contar géneros.
#       Lo omitimos por simplicidad inicial. Si es importante, necesitaría un tratamiento especial.

# Asegúrate de que las columnas a eliminar existen en el DataFrame
existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
if len(existing_cols_to_drop) > 0:
    print(f"\nEliminando columnas: {existing_cols_to_drop}")
    df_processed = df.drop(columns=existing_cols_to_drop)
else:
    df_processed = df.copy()


# Definir variable objetivo (y) y características (X)
target = 'track_popularity'

# Asegúrate de que la columna objetivo existe
if target not in df_processed.columns:
    print(f"Error: La columna objetivo '{target}' no se encuentra en el DataFrame después de eliminar columnas.")
    print(f"Columnas disponibles: {df_processed.columns.tolist()}")
    exit()

y = df_processed[target]
X = df_processed.drop(columns=[target])

# --- 4. Manejo de Valores Faltantes (Imputar con la media) ---
if X.isnull().sum().sum() > 0:
    print("\nSe encontraron valores faltantes en las características. Imputando con la media...")
    X = X.apply(lambda col: col.fillna(col.mean()), axis=0)
    print("Imputación completada.")
else:
    print("\nNo se encontraron valores faltantes en las características seleccionadas.")


print(f"\nVariable Objetivo (y): {target}")
print(f"Características (X) iniciales: {X.columns.tolist()}")

# --- Identificar Tipos de Columnas para Preprocesamiento ---
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

# A veces, columnas como 'key', 'mode', 'time_signature' pueden ser numéricas pero deben tratarse como categóricas
# Ajusta esto según tu dataset específico
potential_cats_numeric = ['key', 'mode', 'time_signature', 'year'] # 'year' puede ser numérico o categórico
for col in potential_cats_numeric:
    if col in numerical_features:
        # Decide si tratarla como categórica
        # Por ejemplo, si 'key' tiene pocos valores únicos (0-11), es categórica.
        if X[col].nunique() < 15: # Umbral arbitrario
             if col not in categorical_features:
                 categorical_features.append(col)
             numerical_features.remove(col)

print(f"\nCaracterísticas Numéricas identificadas: {numerical_features}")
print(f"Características Categóricas identificadas: {categorical_features}")

# --- Crear Pipeline de Preprocesamiento ---
# Para características numéricas: escalar
# Para características categóricas: One-Hot Encode (ignora errores si encuentra categorías no vistas en test)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' # Mantener otras columnas si las hubiera (aunque deberíamos haberlas tratado)
)


# --- Dividir Datos ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nDivisión de datos: {X_train.shape[0]} entrenamiento, {X_test.shape[0]} prueba")




# --- 2. Regresión Lasso con Búsqueda de Alpha ---

# Definir un rango de alphas (logarítmico es común)
alphas_lasso = np.logspace(-4, 1, 100) # Rango de 0.0001 a 10

# Crear pipeline: Preprocesamiento + Modelo LassoCV
# LassoCV usa validación cruzada interna para encontrar el mejor alpha
lasso_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('regression', LassoCV(alphas=alphas_lasso, cv=5, random_state=42, max_iter=10000, tol=0.001)) # cv=5 -> 5-fold CV
])

print("\nEntrenando LassoCV...")
lasso_pipeline.fit(X_train, y_train)

# Mejor alpha encontrado por LassoCV
best_alpha_lasso = lasso_pipeline.named_steps['regression'].alpha_
print(f"Mejor alpha encontrado para Lasso: {best_alpha_lasso:.4f}")

# Evaluar en el conjunto de prueba
y_pred_lasso = lasso_pipeline.predict(X_test)
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
r2_lasso = r2_score(y_test, y_pred_lasso)

print("\n--- Resultados Lasso ---")
print(f"RMSE (Test): {rmse_lasso:.4f}")
print(f"R² (Test): {r2_lasso:.4f}")

# Opcional: Ver coeficientes (Lasso tiende a poner coeficientes a cero)
# Necesitamos obtener los nombres de las características después del OneHotEncoding
try:
    feature_names = numerical_features + \
                    list(lasso_pipeline.named_steps['preprocess']
                         .named_transformers_['cat']
                         .get_feature_names_out(categorical_features))
    lasso_coefs = lasso_pipeline.named_steps['regression'].coef_
    coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': lasso_coefs})
    print("\nCoeficientes Lasso (los no cero son los más 'importantes' según Lasso):")
    print(coef_df[coef_df['Coefficient'] != 0].sort_values(by='Coefficient', key=abs, ascending=False).head(10))
except Exception as e:
    print(f"\nNo se pudieron obtener los nombres de las características para los coeficientes: {e}")


# --- 3. Regresión Ridge con Búsqueda de Alpha ---

# Definir un rango de alphas
alphas_ridge = np.logspace(-3, 4, 100) # Rango de 0.001 a 10000

# Crear pipeline: Preprocesamiento + Modelo RidgeCV
# RidgeCV también usa CV interna
ridge_pipeline = Pipeline([
    ('preprocess', preprocessor),
    # Usamos neg_mean_squared_error porque RidgeCV maximiza el score, y queremos minimizar MSE
    ('regression', RidgeCV(alphas=alphas_ridge, cv=5, scoring='neg_mean_squared_error'))
])

print("\nEntrenando RidgeCV...")
ridge_pipeline.fit(X_train, y_train)

# Mejor alpha encontrado por RidgeCV
best_alpha_ridge = ridge_pipeline.named_steps['regression'].alpha_
print(f"Mejor alpha encontrado para Ridge: {best_alpha_ridge:.4f}")

# Evaluar en el conjunto de prueba
y_pred_ridge = ridge_pipeline.predict(X_test)
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
r2_ridge = r2_score(y_test, y_pred_ridge)

print("\n--- Resultados Ridge ---")
print(f"RMSE (Test): {rmse_ridge:.4f}")
print(f"R² (Test): {r2_ridge:.4f}")

# --- 4. Comparar Resultados ---

print("\n--- Comparación Final (Test Set) ---")
print(f"Lasso: RMSE={rmse_lasso:.4f}, R²={r2_lasso:.4f}, Alpha={best_alpha_lasso:.4f}")
print(f"Ridge: RMSE={rmse_ridge:.4f}, R²={r2_ridge:.4f}, Alpha={best_alpha_ridge:.4f}")

if rmse_lasso < rmse_ridge:
    print("\nLasso parece tener un mejor rendimiento (menor RMSE).")
elif rmse_ridge < rmse_lasso:
     print("\nRidge parece tener un mejor rendimiento (menor RMSE).")
else:
     print("\nLasso y Ridge tienen un rendimiento similar (según RMSE).")

# R²: Más cercano a 1 es mejor.
if r2_lasso > r2_ridge:
    print("Lasso explica una mayor proporción de la varianza (mayor R²).")
elif r2_ridge > r2_lasso:
    print("Ridge explica una mayor proporción de la varianza (mayor R²).")

Dataset cargado. Primeras filas:
                                        playlist_url  year  \
0  https://open.spotify.com/playlist/37i9dQZF1DWU...  2000   
1  https://open.spotify.com/playlist/37i9dQZF1DWU...  2000   
2  https://open.spotify.com/playlist/37i9dQZF1DWU...  2000   
3  https://open.spotify.com/playlist/37i9dQZF1DWU...  2000   
4  https://open.spotify.com/playlist/37i9dQZF1DWU...  2000   

                 track_id            track_name  track_popularity  \
0  3AJwUDP919kvQ9QcozQPxg                Yellow                91   
1  2m1hi0nfMR9vdGC8UcrnwU  All The Small Things                84   
2  3y4LxiYMgDl4RethdzpmNe               Breathe                69   
3  60a0Rd6pjrkxjPbaKzXjfq            In the End                88   
4  62bOmKYxYg7dhrC6gH9vFn           Bye Bye Bye                74   

                           album               artist_id  artist_name  \
0                     Parachutes  4gzpq5DPGxSnKTe4SA8HAU     Coldplay   
1             Enema Of The State 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score
import warnings

# Ignorar advertencias para mantener la salida limpia (opcional)
warnings.filterwarnings('ignore')



df = pd.read_csv(file_path)


# --- 2. Preprocesamiento de Datos ---

# Seleccionar características (features) y variable objetivo (target)
# Excluimos columnas no numéricas o identificadores que no usaremos directamente
# 'artist_genres' es complejo (lista/string), lo excluimos por simplicidad inicial.
# Podría requerir técnicas más avanzadas (como TF-IDF o embeddings) si se quisiera usar.
features = [
    'year', 'artist_popularity', 'danceability', 'energy', 'key',
    'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
    'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'
]
target = 'track_popularity'

# Verificar si todas las columnas necesarias existen
missing_cols = [col for col in features + [target] if col not in df.columns]
if missing_cols:
    print(f"\nError: Faltan las siguientes columnas en el dataset: {missing_cols}")
    exit()

# Crear X (features) e y (target)
X = df[features]
y = df[target]

# Manejar valores faltantes (si los hay) - Estrategia simple: imputar con la media
# Podrías elegir otras estrategias (mediana, eliminar filas, etc.)
if X.isnull().sum().sum() > 0:
    print("\nDetectados valores faltantes en las características. Imputando con la media...")
    for col in X.columns:
        if X[col].isnull().any():
            mean_val = X[col].mean()
            X[col] = X[col].fillna(mean_val)
    print("Imputación completada.")

if y.isnull().sum() > 0:
    print("\nDetectados valores faltantes en la variable objetivo. Eliminando filas...")
    original_len = len(df)
    df_cleaned = df.dropna(subset=[target])
    X = df_cleaned[features]
    y = df_cleaned[target]
    # Re-imputar X por si acaso la eliminación de filas en y introdujo NaNs en X de nuevo (poco probable aquí)
    if X.isnull().sum().sum() > 0:
        for col in X.columns:
             if X[col].isnull().any():
                mean_val = X[col].mean()
                X[col] = X[col].fillna(mean_val)
    print(f"Se eliminaron {original_len - len(df_cleaned)} filas con valor objetivo faltante.")


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")

# Escalar las características numéricas
# Es crucial para modelos regularizados (Lasso, Ridge)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertir de nuevo a DataFrame para facilitar la visualización (opcional)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=features)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=features)

# --- 3. Modelo Base: Regresión Lineal ---
print("\n--- Modelo Base: Regresión Lineal ---")
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Predicciones
y_pred_lr = lr_model.predict(X_test_scaled)

# Evaluación
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print(f"Mean Squared Error (MSE): {mse_lr:.4f}")
print(f"R-squared (R²): {r2_lr:.4f}")

# --- 4. Búsqueda del Mejor Modelo Regularizado (Lasso/Ridge) ---

def find_best_regularized_model(X_train, y_train, X_test, y_test, lasso_alphas, ridge_alphas):
    """
    Entrena modelos Lasso y Ridge con diferentes alphas y encuentra el mejor.

    Args:
        X_train: Características de entrenamiento (escaladas).
        y_train: Variable objetivo de entrenamiento.
        X_test: Características de prueba (escaladas).
        y_test: Variable objetivo de prueba.
        lasso_alphas: Lista o iterable de valores alpha para probar en Lasso.
        ridge_alphas: Lista o iterable de valores alpha para probar en Ridge.

    Returns:
        Un diccionario con la información del mejor modelo encontrado:
        {'model_type': 'Lasso' o 'Ridge', 'best_alpha': mejor valor alpha,
         'best_r2_score': R² en el conjunto de prueba, 'best_mse': MSE en el conjunto de prueba,
         'best_model': el objeto del modelo sklearn entrenado}
    """
    best_model_info = {
        'model_type': None,
        'best_alpha': None,
        'best_r2_score': -np.inf, # Inicializar con un valor muy bajo para R²
        'best_mse': np.inf,      # Inicializar con un valor muy alto para MSE
        'best_model': None
    }

    print("\n--- Buscando Mejor Modelo Regularizado ---")
    print(f"Probando Alphas para Lasso: {list(lasso_alphas)}")
    print(f"Probando Alphas para Ridge: {list(ridge_alphas)}")

    # Probar modelos Lasso
    for alpha in lasso_alphas:
        lasso = Lasso(alpha=alpha, random_state=42, max_iter=10000) # Aumentar max_iter si no converge
        lasso.fit(X_train, y_train)
        y_pred = lasso.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        print(f"  Lasso(alpha={alpha:.2f}): R²={r2:.4f}, MSE={mse:.4f}")

        if r2 > best_model_info['best_r2_score']:
            best_model_info['model_type'] = 'Lasso'
            best_model_info['best_alpha'] = alpha
            best_model_info['best_r2_score'] = r2
            best_model_info['best_mse'] = mse
            best_model_info['best_model'] = lasso

    # Probar modelos Ridge
    for alpha in ridge_alphas:
        ridge = Ridge(alpha=alpha, random_state=42)
        ridge.fit(X_train, y_train)
        y_pred = ridge.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        print(f"  Ridge(alpha={alpha:.2f}): R²={r2:.4f}, MSE={mse:.4f}")

        if r2 > best_model_info['best_r2_score']:
            best_model_info['model_type'] = 'Ridge'
            best_model_info['best_alpha'] = alpha
            best_model_info['best_r2_score'] = r2
            best_model_info['best_mse'] = mse
            best_model_info['best_model'] = ridge

    print("\n--- Mejor Modelo Regularizado Encontrado ---")
    if best_model_info['best_model']:
        print(f"Tipo de Modelo: {best_model_info['model_type']}")
        print(f"Mejor Alpha: {best_model_info['best_alpha']}")
        print(f"Mejor R² (en test): {best_model_info['best_r2_score']:.4f}")
        print(f"Mejor MSE (en test): {best_model_info['best_mse']:.4f}")
    else:
        print("No se encontró un modelo mejor que la inicialización.")

    return best_model_info

# Definir los rangos de alpha a probar según tu solicitud
# Ridge: 2 a 10 (incluidos)
ridge_alpha_range = range(2, 11) # range(start, stop) -> stop is exclusive, so use 11
# Lasso: 0.1, 1, 10
lasso_alpha_range = [0.1, 1.0, 10.0]

# Ejecutar la función de búsqueda
best_regularized_model_info = find_best_regularized_model(
    X_train_scaled, y_train, X_test_scaled, y_test,
    lasso_alpha_range, ridge_alpha_range
)

# --- 5. Análisis de Coeficientes (Opcional) ---
if best_regularized_model_info['best_model']:
    print("\n--- Coeficientes del Mejor Modelo Regularizado ---")
    best_model = best_regularized_model_info['best_model']
    coefficients = pd.DataFrame({
        'Feature': features,
        'Coefficient': best_model.coef_
    })
    # Ordenar por valor absoluto del coeficiente para ver las características más influyentes
    coefficients['Abs_Coefficient'] = coefficients['Coefficient'].abs()
    coefficients = coefficients.sort_values(by='Abs_Coefficient', ascending=False).drop('Abs_Coefficient', axis=1)
    print(coefficients)

    # En Lasso, algunos coeficientes pueden ser exactamente cero
    if best_regularized_model_info['model_type'] == 'Lasso':
        zero_coeffs = coefficients[coefficients['Coefficient'] == 0]
        print(f"\nCaracterísticas con coeficiente cero (eliminadas por Lasso): {len(zero_coeffs)}")
        # print(zero_coeffs['Feature'].tolist()) # Descomentar si quieres ver la lista
else:
    print("\nNo se puede mostrar coeficientes ya que no se seleccionó un mejor modelo regularizado.")


print("\n--- Resumen Final ---")
print(f"Regresión Lineal Base: R²={r2_lr:.4f}, MSE={mse_lr:.4f}")
if best_regularized_model_info['best_model']:
    print(f"Mejor Modelo Regularizado ({best_regularized_model_info['model_type']}, alpha={best_regularized_model_info['best_alpha']}): R²={best_regularized_model_info['best_r2_score']:.4f}, MSE={best_regularized_model_info['best_mse']:.4f}")
    improvement = best_regularized_model_info['best_r2_score'] - r2_lr
    print(f"Mejora en R² respecto a Regresión Lineal: {improvement:+.4f}")
else:
    print("No se encontró un modelo regularizado que mejorara el R² inicial.")


Detectados valores faltantes en las características. Imputando con la media...
Imputación completada.

Tamaño del conjunto de entrenamiento: (1840, 15)
Tamaño del conjunto de prueba: (460, 15)

--- Modelo Base: Regresión Lineal ---
Mean Squared Error (MSE): 193.0950
R-squared (R²): 0.0457

--- Buscando Mejor Modelo Regularizado ---
Probando Alphas para Lasso: [0.1, 1.0, 10.0]
Probando Alphas para Ridge: [2, 3, 4, 5, 6, 7, 8, 9, 10]
  Lasso(alpha=0.10): R²=0.0520, MSE=191.8112
  Lasso(alpha=1.00): R²=0.0552, MSE=191.1774
  Lasso(alpha=10.00): R²=-0.0040, MSE=203.1363
  Ridge(alpha=2.00): R²=0.0457, MSE=193.0837
  Ridge(alpha=3.00): R²=0.0458, MSE=193.0781
  Ridge(alpha=4.00): R²=0.0458, MSE=193.0725
  Ridge(alpha=5.00): R²=0.0458, MSE=193.0669
  Ridge(alpha=6.00): R²=0.0458, MSE=193.0614
  Ridge(alpha=7.00): R²=0.0459, MSE=193.0559
  Ridge(alpha=8.00): R²=0.0459, MSE=193.0504
  Ridge(alpha=9.00): R²=0.0459, MSE=193.0449
  Ridge(alpha=10.00): R²=0.0460, MSE=193.0394

--- Mejor Modelo Re

### **Regresion poli**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import warnings

# Ignorar advertencias para mantener la salida limpia (opcional)
warnings.filterwarnings('ignore')

# --- 1. Carga de Datos ---
# Asegúrate de subir tu archivo CSV a Google Colab y actualizar la ruta
# Ejemplo: '/content/nombre_de_tu_archivo.csv'
file_path = '/content/your_dataset.csv' # <-- ¡CAMBIA ESTA RUTA!

try:
    df = pd.read_csv(file_path)
    print("Dataset cargado exitosamente.")
    print(f"Forma del dataset: {df.shape}")
    # print("\nPrimeras 5 filas:")
    # print(df.head())
    # print("\nInformación del dataset:")
    # df.info() # Descomentar si necesitas ver la info detallada
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta '{file_path}'.")
    print("Por favor, sube tu archivo a Google Colab y verifica la ruta.")
    exit()
except Exception as e:
    print(f"Ocurrió un error al cargar el dataset: {e}")
    exit()

# --- 2. Preprocesamiento de Datos ---

# Seleccionar características (features) y variable objetivo (target)
features = [
    'year', 'artist_popularity', 'danceability', 'energy', 'key',
    'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
    'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'
]
target = 'track_popularity'

# Verificar si todas las columnas necesarias existen
missing_cols = [col for col in features + [target] if col not in df.columns]
if missing_cols:
    print(f"\nError: Faltan las siguientes columnas en el dataset: {missing_cols}")
    exit()

# Eliminar filas con valores faltantes en características o target (estrategia simple)
# Puedes cambiar esto por imputación si lo prefieres
print(f"\nForma original: {df.shape}")
df_cleaned = df.dropna(subset=features + [target])
print(f"Forma después de eliminar NaNs: {df_cleaned.shape}")
print(f"Se eliminaron {df.shape[0] - df_cleaned.shape[0]} filas con valores faltantes.")

if df_cleaned.empty:
    print("\nError: El dataset quedó vacío después de eliminar filas con NaNs.")
    exit()

X = df_cleaned[features]
y = df_cleaned[target]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")

# --- 3. Modelo Base: Regresión Lineal (Grado Polinómico 1) ---
print("\n--- Modelo Base: Regresión Lineal Simple ---")
# Usamos un pipeline para escalar y luego aplicar regresión lineal
# Aunque no es estrictamente necesario para LR simple, mantiene la consistencia
base_lr_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_regression', LinearRegression())
])

base_lr_pipe.fit(X_train, y_train)
y_pred_lr_base = base_lr_pipe.predict(X_test)
mse_lr_base = mean_squared_error(y_test, y_pred_lr_base)
r2_lr_base = r2_score(y_test, y_pred_lr_base)

print(f"Mean Squared Error (MSE): {mse_lr_base:.4f}")
print(f"R-squared (R²): {r2_lr_base:.4f}")

# --- 4. Función para Buscar el Mejor Modelo Polinómico Regularizado ---

def find_best_polynomial_regularized_model(X_train, y_train, X_test, y_test, degrees, alphas):
    """
    Prueba modelos Lasso y Ridge con diferentes grados polinómicos y alphas.

    Args:
        X_train: Características de entrenamiento (sin escalar).
        y_train: Variable objetivo de entrenamiento.
        X_test: Características de prueba (sin escalar).
        y_test: Variable objetivo de prueba.
        degrees: Lista o iterable de grados polinómicos a probar (ej: range(1, 11)).
        alphas: Lista o iterable de valores alpha a probar (ej: [0.1, 1, 10]).

    Returns:
        Un diccionario con la información del mejor modelo encontrado.
    """
    best_model_info = {
        'model_type': None,
        'degree': None,
        'alpha': None,
        'best_r2_score': -np.inf, # Inicializar R² con valor muy bajo
        'best_mse': np.inf,      # Inicializar MSE con valor muy alto
        'best_pipeline': None    # Guardaremos el pipeline completo
    }

    print("\n--- Buscando Mejor Modelo Polinómico Regularizado ---")
    print(f"Probando Grados Polinómicos: {list(degrees)}")
    print(f"Probando Alphas: {list(alphas)}")

    for degree in degrees:
        print(f"\n--- Probando Grado Polinómico: {degree} ---")

        # Crear el transformador de características polinómicas
        # include_bias=False porque el modelo lineal (Lasso/Ridge) ya maneja la intercepción
        poly_features = PolynomialFeatures(degree=degree, include_bias=False)

        # Probar Ridge con este grado
        for alpha in alphas:
            # Crear el pipeline completo: Poly -> Scaler -> Ridge
            ridge_pipe = Pipeline([
                ('poly', poly_features),
                ('scaler', StandardScaler()),
                ('ridge', Ridge(alpha=alpha, random_state=42))
            ])

            try:
                # Entrenar el pipeline
                ridge_pipe.fit(X_train, y_train)
                # Predecir en el conjunto de prueba
                y_pred = ridge_pipe.predict(X_test)
                # Evaluar
                r2 = r2_score(y_test, y_pred)
                mse = mean_squared_error(y_test, y_pred)
                print(f"  Grado={degree}, Ridge(alpha={alpha:.2f}): R²={r2:.4f}, MSE={mse:.4f}")

                # Actualizar si es el mejor modelo hasta ahora
                if r2 > best_model_info['best_r2_score']:
                    best_model_info['model_type'] = 'Ridge'
                    best_model_info['degree'] = degree
                    best_model_info['alpha'] = alpha
                    best_model_info['best_r2_score'] = r2
                    best_model_info['best_mse'] = mse
                    best_model_info['best_pipeline'] = ridge_pipe

            except MemoryError:
                 print(f"  Grado={degree}, Ridge(alpha={alpha:.2f}): Error de Memoria - Demasiadas características polinómicas. Saltando.")
                 # Si un grado causa error de memoria, probablemente los siguientes también lo harán para este alpha
                 # Podríamos romper el bucle interno de alpha aquí si es necesario
                 continue # Pasar al siguiente alpha o grado
            except Exception as e:
                 print(f"  Grado={degree}, Ridge(alpha={alpha:.2f}): Error inesperado - {e}. Saltando.")
                 continue

        # Probar Lasso con este grado
        for alpha in alphas:
             # Crear el pipeline completo: Poly -> Scaler -> Lasso
            lasso_pipe = Pipeline([
                ('poly', poly_features),
                ('scaler', StandardScaler()),
                ('lasso', Lasso(alpha=alpha, random_state=42, max_iter=10000, tol=0.01)) # Aumentar max_iter y tol puede ayudar a la convergencia
            ])

            try:
                # Entrenar el pipeline
                lasso_pipe.fit(X_train, y_train)
                # Predecir en el conjunto de prueba
                y_pred = lasso_pipe.predict(X_test)
                # Evaluar
                r2 = r2_score(y_test, y_pred)
                mse = mean_squared_error(y_test, y_pred)
                print(f"  Grado={degree}, Lasso(alpha={alpha:.2f}): R²={r2:.4f}, MSE={mse:.4f}")

                # Actualizar si es el mejor modelo hasta ahora
                if r2 > best_model_info['best_r2_score']:
                    best_model_info['model_type'] = 'Lasso'
                    best_model_info['degree'] = degree
                    best_model_info['alpha'] = alpha
                    best_model_info['best_r2_score'] = r2
                    best_model_info['best_mse'] = mse
                    best_model_info['best_pipeline'] = lasso_pipe

            except MemoryError:
                 print(f"  Grado={degree}, Lasso(alpha={alpha:.2f}): Error de Memoria - Demasiadas características polinómicas. Saltando.")
                 continue
            except Exception as e:
                 print(f"  Grado={degree}, Lasso(alpha={alpha:.2f}): Error inesperado - {e}. Saltando.")
                 continue


    print("\n--- Mejor Modelo Polinómico Regularizado Encontrado ---")
    if best_model_info['best_pipeline']:
        print(f"Tipo de Modelo: {best_model_info['model_type']}")
        print(f"Grado Polinómico: {best_model_info['degree']}")
        print(f"Mejor Alpha: {best_model_info['alpha']}")
        print(f"Mejor R² (en test): {best_model_info['best_r2_score']:.4f}")
        print(f"Mejor MSE (en test): {best_model_info['best_mse']:.4f}")
    else:
        print("No se encontró un modelo que mejorara la inicialización (posiblemente debido a errores).")

    return best_model_info

# --- 5. Ejecutar la Búsqueda ---

# Definir los rangos a probar según tu solicitud
degrees_to_test = range(1, 11) # Grados del 1 al 10
alphas_to_test = [0.1, 1.0, 10.0] # Alphas 0.1, 1, 10

# Ejecutar la función de búsqueda
# Pasamos los datos SIN escalar, ya que el escalado está dentro del pipeline
best_poly_reg_model_info = find_best_polynomial_regularized_model(
    X_train, y_train, X_test, y_test,
    degrees_to_test, alphas_to_test
)

# --- 6. Resumen Final ---
print("\n--- Resumen Final de Rendimiento (en conjunto de prueba) ---")
print(f"Regresión Lineal Simple (Grado 1): R²={r2_lr_base:.4f}, MSE={mse_lr_base:.4f}")

if best_poly_reg_model_info['best_pipeline']:
    print(f"Mejor Modelo Encontrado ({best_poly_reg_model_info['model_type']}, Grado={best_poly_reg_model_info['degree']}, Alpha={best_poly_reg_model_info['alpha']}): R²={best_poly_reg_model_info['best_r2_score']:.4f}, MSE={best_poly_reg_model_info['best_mse']:.4f}")
    improvement_r2 = best_poly_reg_model_info['best_r2_score'] - r2_lr_base
    print(f"Mejora en R² respecto a Regresión Lineal Simple: {improvement_r2:+.4f}")
else:
    print("No se encontró un modelo polinómico regularizado óptimo en la búsqueda (comparado con R²=-inf).")

# --- 7. (Opcional) Análisis de Coeficientes del Mejor Modelo ---
# La interpretación es más compleja con características polinómicas
if best_poly_reg_model_info['best_pipeline']:
    best_pipeline = best_poly_reg_model_info['best_pipeline']
    try:
        # Obtener los nombres de las características polinómicas generadas
        poly_step = best_pipeline.named_steps['poly']
        model_step = best_pipeline.steps[-1][1] # El último paso es el modelo (Lasso o Ridge)

        feature_names_poly = poly_step.get_feature_names_out(input_features=features)

        coefficients = pd.DataFrame({
            'Feature': feature_names_poly,
            'Coefficient': model_step.coef_
        })
        coefficients['Abs_Coefficient'] = coefficients['Coefficient'].abs()
        coefficients = coefficients.sort_values(by='Abs_Coefficient', ascending=False).drop('Abs_Coefficient', axis=1)

        print(f"\n--- Coeficientes del Mejor Modelo ({best_poly_reg_model_info['model_type']}, Grado={best_poly_reg_model_info['degree']}, Alpha={best_poly_reg_model_info['alpha']}) ---")
        print(coefficients.head(15)) # Mostrar solo los N más importantes por claridad

        if best_poly_reg_model_info['model_type'] == 'Lasso':
            zero_coeffs = coefficients[np.isclose(coefficients['Coefficient'], 0)] # Usar isclose por precisión flotante
            print(f"\nCaracterísticas con coeficiente cercano a cero (potencialmente eliminadas por Lasso): {len(zero_coeffs)}")

    except Exception as e:
        print(f"\nNo se pudieron extraer los coeficientes del mejor modelo: {e}")

Error: No se encontró el archivo en la ruta '/content/your_dataset.csv'.
Por favor, sube tu archivo a Google Colab y verifica la ruta.

Forma original: (2300, 23)
Forma después de eliminar NaNs: (2299, 23)
Se eliminaron 1 filas con valores faltantes.

Tamaño del conjunto de entrenamiento: (1839, 15)
Tamaño del conjunto de prueba: (460, 15)

--- Modelo Base: Regresión Lineal Simple ---
Mean Squared Error (MSE): 148.2121
R-squared (R²): 0.0832

--- Buscando Mejor Modelo Polinómico Regularizado ---
Probando Grados Polinómicos: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Probando Alphas: [0.1, 1.0, 10.0]

--- Probando Grado Polinómico: 1 ---
  Grado=1, Ridge(alpha=0.10): R²=0.0832, MSE=148.2118
  Grado=1, Ridge(alpha=1.00): R²=0.0833, MSE=148.2092
  Grado=1, Ridge(alpha=10.00): R²=0.0834, MSE=148.1830
  Grado=1, Lasso(alpha=0.10): R²=0.0860, MSE=147.7604
  Grado=1, Lasso(alpha=1.00): R²=0.0808, MSE=148.6134
  Grado=1, Lasso(alpha=10.00): R²=-0.0000, MSE=161.6703

--- Probando Grado Polinómico: 2 ---
 

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import warnings

# Ignorar advertencias para mantener la salida limpia (opcional)
warnings.filterwarnings('ignore')



data = pd.read_csv(file_path)
df = data.copy()

# --- 2. Preprocesamiento de Datos ---

# Seleccionar características (features) y variable objetivo (target)
features = [
    'year', 'artist_popularity', 'danceability', 'energy', 'key',
    'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
    'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'
]
target = 'track_popularity'

# Verificar si todas las columnas necesarias existen
missing_cols = [col for col in features + [target] if col not in df.columns]
if missing_cols:
    print(f"\nError: Faltan las siguientes columnas en el dataset: {missing_cols}")
    exit()

# Eliminar filas con valores faltantes en características o target (estrategia simple)
# Puedes cambiar esto por imputación si lo prefieres
print(f"\nForma original: {df.shape}")
df_cleaned = df.dropna(subset=features + [target])
print(f"Forma después de eliminar NaNs: {df_cleaned.shape}")
print(f"Se eliminaron {df.shape[0] - df_cleaned.shape[0]} filas con valores faltantes.")

if df_cleaned.empty:
    print("\nError: El dataset quedó vacío después de eliminar filas con NaNs.")
    exit()

X = df_cleaned[features]
y = df_cleaned[target]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")

# --- 3. Modelo Base: Regresión Lineal (Grado Polinómico 1) ---
print("\n--- Modelo Base: Regresión Lineal Simple ---")
# Usamos un pipeline para escalar y luego aplicar regresión lineal
# Aunque no es estrictamente necesario para LR simple, mantiene la consistencia
base_lr_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_regression', LinearRegression())
])

base_lr_pipe.fit(X_train, y_train)
y_pred_lr_base = base_lr_pipe.predict(X_test)
mse_lr_base = mean_squared_error(y_test, y_pred_lr_base)
r2_lr_base = r2_score(y_test, y_pred_lr_base)

print(f"Mean Squared Error (MSE): {mse_lr_base:.4f}")
print(f"R-squared (R²): {r2_lr_base:.4f}")

# --- 4. Función para Buscar el Mejor Modelo Polinómico Regularizado ---

def find_best_polynomial_regularized_model(X_train, y_train, X_test, y_test, degrees, alphas):
    """
    Prueba modelos Lasso y Ridge con diferentes grados polinómicos y alphas.

    Args:
        X_train: Características de entrenamiento (sin escalar).
        y_train: Variable objetivo de entrenamiento.
        X_test: Características de prueba (sin escalar).
        y_test: Variable objetivo de prueba.
        degrees: Lista o iterable de grados polinómicos a probar (ej: range(1, 11)).
        alphas: Lista o iterable de valores alpha a probar (ej: [0.1, 1, 10]).

    Returns:
        Un diccionario con la información del mejor modelo encontrado.
    """
    best_model_info = {
        'model_type': None,
        'degree': None,
        'alpha': None,
        'best_r2_score': -np.inf, # Inicializar R² con valor muy bajo
        'best_mse': np.inf,      # Inicializar MSE con valor muy alto
        'best_pipeline': None    # Guardaremos el pipeline completo
    }

    print("\n--- Buscando Mejor Modelo Polinómico Regularizado ---")
    print(f"Probando Grados Polinómicos: {list(degrees)}")
    print(f"Probando Alphas: {list(alphas)}")

    for degree in degrees:
        print(f"\n--- Probando Grado Polinómico: {degree} ---")

        # Crear el transformador de características polinómicas
        # include_bias=False porque el modelo lineal (Lasso/Ridge) ya maneja la intercepción
        poly_features = PolynomialFeatures(degree=degree, include_bias=False)

        # Probar Ridge con este grado
        for alpha in alphas:
            # Crear el pipeline completo: Poly -> Scaler -> Ridge
            ridge_pipe = Pipeline([
                ('poly', poly_features),
                ('scaler', StandardScaler()),
                ('ridge', Ridge(alpha=alpha, random_state=42))
            ])

            try:
                # Entrenar el pipeline
                ridge_pipe.fit(X_train, y_train)
                # Predecir en el conjunto de prueba
                y_pred = ridge_pipe.predict(X_test)
                # Evaluar
                r2 = r2_score(y_test, y_pred)
                mse = mean_squared_error(y_test, y_pred)
                print(f"  Grado={degree}, Ridge(alpha={alpha:.2f}): R²={r2:.4f}, MSE={mse:.4f}")

                # Actualizar si es el mejor modelo hasta ahora
                if r2 > best_model_info['best_r2_score']:
                    best_model_info['model_type'] = 'Ridge'
                    best_model_info['degree'] = degree
                    best_model_info['alpha'] = alpha
                    best_model_info['best_r2_score'] = r2
                    best_model_info['best_mse'] = mse
                    best_model_info['best_pipeline'] = ridge_pipe

            except MemoryError:
                 print(f"  Grado={degree}, Ridge(alpha={alpha:.2f}): Error de Memoria - Demasiadas características polinómicas. Saltando.")
                 # Si un grado causa error de memoria, probablemente los siguientes también lo harán para este alpha
                 # Podríamos romper el bucle interno de alpha aquí si es necesario
                 continue # Pasar al siguiente alpha o grado
            except Exception as e:
                 print(f"  Grado={degree}, Ridge(alpha={alpha:.2f}): Error inesperado - {e}. Saltando.")
                 continue

        # Probar Lasso con este grado
        for alpha in alphas:
             # Crear el pipeline completo: Poly -> Scaler -> Lasso
            lasso_pipe = Pipeline([
                ('poly', poly_features),
                ('scaler', StandardScaler()),
                ('lasso', Lasso(alpha=alpha, random_state=42, max_iter=10000, tol=0.01)) # Aumentar max_iter y tol puede ayudar a la convergencia
            ])

            try:
                # Entrenar el pipeline
                lasso_pipe.fit(X_train, y_train)
                # Predecir en el conjunto de prueba
                y_pred = lasso_pipe.predict(X_test)
                # Evaluar
                r2 = r2_score(y_test, y_pred)
                mse = mean_squared_error(y_test, y_pred)
                print(f"  Grado={degree}, Lasso(alpha={alpha:.2f}): R²={r2:.4f}, MSE={mse:.4f}")

                # Actualizar si es el mejor modelo hasta ahora
                if r2 > best_model_info['best_r2_score']:
                    best_model_info['model_type'] = 'Lasso'
                    best_model_info['degree'] = degree
                    best_model_info['alpha'] = alpha
                    best_model_info['best_r2_score'] = r2
                    best_model_info['best_mse'] = mse
                    best_model_info['best_pipeline'] = lasso_pipe

            except MemoryError:
                 print(f"  Grado={degree}, Lasso(alpha={alpha:.2f}): Error de Memoria - Demasiadas características polinómicas. Saltando.")
                 continue
            except Exception as e:
                 print(f"  Grado={degree}, Lasso(alpha={alpha:.2f}): Error inesperado - {e}. Saltando.")
                 continue


    print("\n--- Mejor Modelo Polinómico Regularizado Encontrado ---")
    if best_model_info['best_pipeline']:
        print(f"Tipo de Modelo: {best_model_info['model_type']}")
        print(f"Grado Polinómico: {best_model_info['degree']}")
        print(f"Mejor Alpha: {best_model_info['alpha']}")
        print(f"Mejor R² (en test): {best_model_info['best_r2_score']:.4f}")
        print(f"Mejor MSE (en test): {best_model_info['best_mse']:.4f}")
    else:
        print("No se encontró un modelo que mejorara la inicialización (posiblemente debido a errores).")

    return best_model_info

# --- 5. Ejecutar la Búsqueda ---

# Definir los rangos a probar según tu solicitud
degrees_to_test = range(1, 11) # Grados del 1 al 10
alphas_to_test = [0.1, 1.0, 10.0] # Alphas 0.1, 1, 10

# Ejecutar la función de búsqueda
# Pasamos los datos SIN escalar, ya que el escalado está dentro del pipeline
best_poly_reg_model_info = find_best_polynomial_regularized_model(
    X_train, y_train, X_test, y_test,
    degrees_to_test, alphas_to_test
)

# --- 6. Resumen Final ---
print("\n--- Resumen Final de Rendimiento (en conjunto de prueba) ---")
print(f"Regresión Lineal Simple (Grado 1): R²={r2_lr_base:.4f}, MSE={mse_lr_base:.4f}")

if best_poly_reg_model_info['best_pipeline']:
    print(f"Mejor Modelo Encontrado ({best_poly_reg_model_info['model_type']}, Grado={best_poly_reg_model_info['degree']}, Alpha={best_poly_reg_model_info['alpha']}): R²={best_poly_reg_model_info['best_r2_score']:.4f}, MSE={best_poly_reg_model_info['best_mse']:.4f}")
    improvement_r2 = best_poly_reg_model_info['best_r2_score'] - r2_lr_base
    print(f"Mejora en R² respecto a Regresión Lineal Simple: {improvement_r2:+.4f}")
else:
    print("No se encontró un modelo polinómico regularizado óptimo en la búsqueda (comparado con R²=-inf).")

# --- 7. (Opcional) Análisis de Coeficientes del Mejor Modelo ---
# La interpretación es más compleja con características polinómicas
if best_poly_reg_model_info['best_pipeline']:
    best_pipeline = best_poly_reg_model_info['best_pipeline']
    try:
        # Obtener los nombres de las características polinómicas generadas
        poly_step = best_pipeline.named_steps['poly']
        model_step = best_pipeline.steps[-1][1] # El último paso es el modelo (Lasso o Ridge)

        feature_names_poly = poly_step.get_feature_names_out(input_features=features)

        coefficients = pd.DataFrame({
            'Feature': feature_names_poly,
            'Coefficient': model_step.coef_
        })
        coefficients['Abs_Coefficient'] = coefficients['Coefficient'].abs()
        coefficients = coefficients.sort_values(by='Abs_Coefficient', ascending=False).drop('Abs_Coefficient', axis=1)

        print(f"\n--- Coeficientes del Mejor Modelo ({best_poly_reg_model_info['model_type']}, Grado={best_poly_reg_model_info['degree']}, Alpha={best_poly_reg_model_info['alpha']}) ---")
        print(coefficients.head(15)) # Mostrar solo los N más importantes por claridad

        if best_poly_reg_model_info['model_type'] == 'Lasso':
            zero_coeffs = coefficients[np.isclose(coefficients['Coefficient'], 0)] # Usar isclose por precisión flotante
            print(f"\nCaracterísticas con coeficiente cercano a cero (potencialmente eliminadas por Lasso): {len(zero_coeffs)}")

    except Exception as e:
        print(f"\nNo se pudieron extraer los coeficientes del mejor modelo: {e}")


Forma original: (2300, 23)
Forma después de eliminar NaNs: (2299, 23)
Se eliminaron 1 filas con valores faltantes.

Tamaño del conjunto de entrenamiento: (1839, 15)
Tamaño del conjunto de prueba: (460, 15)

--- Modelo Base: Regresión Lineal Simple ---
Mean Squared Error (MSE): 148.2121
R-squared (R²): 0.0832

--- Buscando Mejor Modelo Polinómico Regularizado ---
Probando Grados Polinómicos: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Probando Alphas: [0.1, 1.0, 10.0]

--- Probando Grado Polinómico: 1 ---
  Grado=1, Ridge(alpha=0.10): R²=0.0832, MSE=148.2118
  Grado=1, Ridge(alpha=1.00): R²=0.0833, MSE=148.2092
  Grado=1, Ridge(alpha=10.00): R²=0.0834, MSE=148.1830
  Grado=1, Lasso(alpha=0.10): R²=0.0860, MSE=147.7604
  Grado=1, Lasso(alpha=1.00): R²=0.0808, MSE=148.6134
  Grado=1, Lasso(alpha=10.00): R²=-0.0000, MSE=161.6703

--- Probando Grado Polinómico: 2 ---
  Grado=2, Ridge(alpha=0.10): R²=0.0738, MSE=149.7313
  Grado=2, Ridge(alpha=1.00): R²=0.0763, MSE=149.3426
  Grado=2, Ridge(alpha=10.00

In [ ]:
import pandas as pd
import numpy as np
import ast # Para convertir string de lista a lista real
from collections import Counter # Para contar frecuencias de géneros
import warnings

# Preprocesamiento y Modelado
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, KBinsDiscretizer
from sklearn.linear_model import Lasso, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.exceptions import ConvergenceWarning

# --- Configuración Inicial ---
# Ignorar advertencias (opcional pero útil para salidas limpias)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

data = pd.read_csv(file_path)
df = data.copy()

# Seleccionar columnas relevantes iniciales (incluyendo genres)
initial_features = [
    'track_popularity', 'year', 'artist_popularity', 'danceability',
    'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
    'time_signature', 'artist_genres' # Incluimos géneros
]

existing_features = [col for col in initial_features if col in df.columns]
if 'track_popularity' not in existing_features:
    print("Error: La columna objetivo 'track_popularity' no se encuentra.")
    exit()
print(f"\nColumnas iniciales seleccionadas: {existing_features}")

df_subset = df[existing_features].copy()

# Convertir columnas numéricas (excluyendo genres por ahora)
numeric_cols_to_check = [col for col in existing_features if col != 'artist_genres' and col != 'track_popularity']
for col in numeric_cols_to_check:
    df_subset[col] = pd.to_numeric(df_subset[col], errors='coerce')

# Manejar valores faltantes en columnas numéricas y target (eliminar filas)
cols_to_check_na = [col for col in existing_features if col != 'artist_genres']
initial_rows = df_subset.shape[0]
print(f"\nValores faltantes ANTES de limpiar (en subset inicial):\n{df_subset[cols_to_check_na].isnull().sum().sort_values(ascending=False)}")
df_subset.dropna(subset=cols_to_check_na, inplace=True)
print(f"Se eliminaron {initial_rows - df_subset.shape[0]} filas debido a NaNs en columnas numéricas/target.")
print(f"Forma del dataset DESPUÉS de limpiar NaNs numéricos: {df_subset.shape}")

# Asegurarse de que 'artist_genres' sea string y manejar NaNs (reemplazar con '[]')
if 'artist_genres' in df_subset.columns:
    df_subset['artist_genres'] = df_subset['artist_genres'].fillna('[]').astype(str)
else:
    print("Advertencia: La columna 'artist_genres' no se encontró.")

if df_subset.empty:
    print("Error: El dataset quedó vacío después de la limpieza inicial.")
    exit()

# Separar X e y TEMPORALMENTE para aplicar ingeniería solo a X
X_temp = df_subset.drop('track_popularity', axis=1)
y = df_subset['track_popularity'].copy() # y ya está limpio y listo

print("\n--- Iniciando Ingeniería de Características ---")

# Crear una copia para no modificar X_temp directamente en cada paso
X_engineered = X_temp.copy()

# --- 2.1 Procesamiento de Géneros (artist_genres) ---
if 'artist_genres' in X_engineered.columns:
    print("\nProcesando 'artist_genres'...")
    # Función segura para convertir string de lista a lista
    def parse_genre_list(genres_str):
        try:
            genres = ast.literal_eval(genres_str)
            return genres if isinstance(genres, list) else []
        except (ValueError, SyntaxError):
            return []

    X_engineered['genre_list'] = X_engineered['artist_genres'].apply(parse_genre_list)
    all_genres = [genre for sublist in X_engineered['genre_list'] for genre in sublist]
    genre_counts = Counter(all_genres)
    N_TOP_GENRES = 20 # Puedes ajustar esto
    top_genres = [genre for genre, count in genre_counts.most_common(N_TOP_GENRES)]
    print(f"Top {N_TOP_GENRES} géneros encontrados (se crearán columnas OHE): {top_genres}")

    for genre in top_genres:
        col_name = f'genre_{genre.replace(" ", "_").replace("-", "_").replace("&","and").replace("'","")}' # Limpiar nombres
        X_engineered[col_name] = X_engineered['genre_list'].apply(lambda x: 1 if genre in x else 0)

    X_engineered.drop(['artist_genres', 'genre_list'], axis=1, inplace=True)
    print(f"Se añadieron {len(top_genres)} columnas de género (One-Hot Encoded).")
else:
    print("\n'artist_genres' no presente, saltando procesamiento de géneros.")


# --- 2.2 Transformaciones No Lineales ---
print("\nAplicando transformaciones no lineales...")
log_features = ['duration_ms', 'speechiness', 'liveness', 'instrumentalness']
for col in log_features:
    if col in X_engineered.columns:
        if (X_engineered[col] < 0).any():
            print(f"Advertencia: La columna '{col}' contiene valores negativos. No se aplicará log.")
        else:
            X_engineered[f'{col}_log'] = np.log1p(X_engineered[col])
            print(f" - Columna '{col}_log' creada.")
            # X_engineered.drop(col, axis=1, inplace=True) # Decidimos mantener la original

poly_features = ['artist_popularity', 'danceability', 'loudness', 'energy', 'valence']
for col in poly_features:
     if col in X_engineered.columns:
        X_engineered[f'{col}_sq'] = X_engineered[col]**2
        print(f" - Columna '{col}_sq' creada.")
        # X_engineered.drop(col, axis=1, inplace=True) # Mantenemos la original


# --- 2.3 Creación de Interacciones ---
print("\nCreando características de interacción...")
interactions = [
    ('danceability', 'energy'),
    ('valence', 'energy'),
    # ('artist_popularity', 'acousticness') # Añade o quita según creas necesario
]
for col1, col2 in interactions:
    if col1 in X_engineered.columns and col2 in X_engineered.columns:
        X_engineered[f'{col1}_x_{col2}'] = X_engineered[col1] * X_engineered[col2]
        print(f" - Interacción '{col1}_x_{col2}' creada.")

# --- 2.4 Binning (Agrupación) ---
print("\nAplicando binning a 'artist_popularity'...")
if 'artist_popularity' in X_engineered.columns:
    N_BINS = 5
    bin_col_name = 'artist_pop_binned' # Nombre más corto
    try:
        # Usar solo valores finitos para el discretizer
        valid_artist_pop = X_engineered[['artist_popularity']].replace([np.inf, -np.inf], np.nan).dropna()
        if not valid_artist_pop.empty:
            discretizer = KBinsDiscretizer(n_bins=N_BINS, encode='ordinal', strategy='quantile', subsample=None) # subsample=None para usar todos los datos
            # Asegurar que los índices coincidan después de fit_transform
            binned_values = discretizer.fit_transform(valid_artist_pop)
            binned_series = pd.Series(binned_values.flatten(), index=valid_artist_pop.index, name=bin_col_name)

            # Unir los binned values de vuelta al dataframe principal X_engineered
            X_engineered = X_engineered.join(binned_series)
            print(f" - Columna ordinal '{bin_col_name}' creada con {N_BINS} bins.")

            # Llenar NaNs en la columna binarizada (si la popularidad original era NaN) con un valor (e.g., -1 o la media/mediana del bin)
            X_engineered[bin_col_name].fillna(-1, inplace=True) # Usar -1 para categoría 'missing/otro'

            # One-Hot Encode la columna binarizada (incluyendo el -1 si existe)
            ohe_binned = pd.get_dummies(X_engineered[bin_col_name], prefix=bin_col_name, dummy_na=False) # dummy_na=False si ya llenaste NaNs
            X_engineered = pd.concat([X_engineered, ohe_binned], axis=1)
            X_engineered.drop(bin_col_name, axis=1, inplace=True) # Eliminar columna ordinal
            print(f" - Columnas One-Hot creadas para '{bin_col_name}'.")

            # Eliminar la 'artist_popularity' original
            if 'artist_popularity' in X_engineered.columns:
                 X_engineered.drop('artist_popularity', axis=1, inplace=True)
                 print(f" - Columna 'artist_popularity' original eliminada.")
        else:
            print(" - No hay valores válidos en 'artist_popularity' para binarizar. Saltando binning.")

    except ValueError as ve:
         print(f"Error durante el binning de 'artist_popularity' (posiblemente pocos datos o valores repetidos): {ve}. Saltando este paso.")
    except Exception as e:
        print(f"Error inesperado durante el binning de 'artist_popularity': {e}. Saltando este paso.")

else:
    print("'artist_popularity' no encontrada, saltando binning.")


# --- 2.5 Finalización de Ingeniería ---
print("\n--- Ingeniería de Características Completada ---")

# Eliminar columnas que puedan contener infinitos o NaNs residuales
X_engineered.replace([np.inf, -np.inf], np.nan, inplace=True)
if X_engineered.isnull().sum().sum() > 0:
    print(f"\nValores NaN encontrados DESPUÉS de ingeniería:\n{X_engineered.isnull().sum()[X_engineered.isnull().sum() > 0]}")
    print("Imputando NaNs restantes con la mediana de cada columna...")
    for col in X_engineered.columns[X_engineered.isnull().any()]:
        median_val = X_engineered[col].median()
        X_engineered[col].fillna(median_val, inplace=True)

# Asegurarse de que todas las columnas sean numéricas
non_numeric_cols = X_engineered.select_dtypes(exclude=np.number).columns
if len(non_numeric_cols) > 0:
    print(f"\nAdvertencia: Se encontraron columnas no numéricas al final: {list(non_numeric_cols)}. Intentando convertir...")
    for col in non_numeric_cols:
        X_engineered[col] = pd.to_numeric(X_engineered[col], errors='coerce')
    # Volver a imputar si la conversión creó NaNs
    if X_engineered.isnull().sum().sum() > 0:
         for col in X_engineered.columns[X_engineered.isnull().any()]:
            median_val = X_engineered[col].median()
            X_engineered[col].fillna(median_val, inplace=True)

print(f"\nForma final de X (características ingenierizadas): {X_engineered.shape}")
print(f"Número de características finales: {X_engineered.shape[1]}")
# print("\nColumnas finales en X_engineered:") # Descomentar si quieres ver la lista completa
# print(X_engineered.columns.tolist())

# --- 3. División Train/Test (DESPUÉS de la ingeniería) ---
X_train, X_test, y_train, y_test = train_test_split(X_engineered, y, test_size=0.2, random_state=42)

print("\n--- Datos divididos para modelado ---")
print(f"Tamaño Entrenamiento: X={X_train.shape}, y={y_train.shape}")
print(f"Tamaño Prueba: X={X_test.shape}, y={y_test.shape}")


# --- 4. Definición de Pipelines y Búsqueda de Hiperparámetros ---

# Rangos de hiperparámetros a probar
degrees = list(range(2, 11)) # Grados polinómicos de 2 a 10
alphas = [0.1, 1.0, 10.0]    # Alphas para Lasso y Ridge

# Crear pipelines (Scaler -> Poly -> Regressor)
# NOTA: PolynomialFeatures con muchas características de entrada puede ser MUY costoso computacionalmente.
# Si el proceso es demasiado lento, considera reducir el número de características de entrada o el rango de 'degrees'.

# Pipeline para Lasso
pipeline_lasso = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('regressor', Lasso(random_state=42, max_iter=15000)) # Aumentar max_iter puede ser necesario
])

# Pipeline para Ridge
pipeline_ridge = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('regressor', Ridge(random_state=42, solver='auto')) # 'auto' elige el solver más eficiente
])

# Parámetros para GridSearchCV
param_grid = {
    'poly__degree': degrees,
    'regressor__alpha': alphas
}

# Configurar Validación Cruzada
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# --- Ejecutar GridSearchCV para Lasso ---
print("\n--- Iniciando búsqueda de hiperparámetros para Regresión Polinómica con Lasso ---")
grid_search_lasso = GridSearchCV(
    pipeline_lasso,
    param_grid,
    cv=cv,
    scoring='neg_mean_squared_error',
    n_jobs=-1, # Usar todos los procesadores
    verbose=1
)
grid_search_lasso.fit(X_train, y_train) # Entrenar sobre el conjunto de entrenamiento

print("\n--- Resultados de GridSearchCV para Lasso ---")
print(f"Mejor puntuación (neg MSE): {grid_search_lasso.best_score_:.4f}")
best_rmse_cv_lasso = np.sqrt(-grid_search_lasso.best_score_)
print(f"Mejor RMSE (Validación Cruzada): {best_rmse_cv_lasso:.4f}")
print(f"Mejores parámetros: {grid_search_lasso.best_params_}")

# --- Ejecutar GridSearchCV para Ridge ---
print("\n--- Iniciando búsqueda de hiperparámetros para Regresión Polinómica con Ridge ---")
grid_search_ridge = GridSearchCV(
    pipeline_ridge,
    param_grid, # Usamos el mismo grid de grados y alphas
    cv=cv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)
grid_search_ridge.fit(X_train, y_train) # Entrenar sobre el conjunto de entrenamiento

print("\n--- Resultados de GridSearchCV para Ridge ---")
print(f"Mejor puntuación (neg MSE): {grid_search_ridge.best_score_:.4f}")
best_rmse_cv_ridge = np.sqrt(-grid_search_ridge.best_score_)
print(f"Mejor RMSE (Validación Cruzada): {best_rmse_cv_ridge:.4f}")
print(f"Mejores parámetros: {grid_search_ridge.best_params_}")


# --- 5. Selección del Mejor Modelo y Evaluación Final ---
print("\n--- Comparando Modelos Finales ---")

best_model = None
best_model_name = ""
best_params = None
best_cv_rmse = float('inf')

if best_rmse_cv_lasso < best_rmse_cv_ridge:
    print(f"El mejor modelo (basado en CV RMSE) es Lasso Polinómico con RMSE = {best_rmse_cv_lasso:.4f}")
    best_model = grid_search_lasso.best_estimator_
    best_model_name = "Lasso Polinómico"
    best_params = grid_search_lasso.best_params_
    best_cv_rmse = best_rmse_cv_lasso
else:
    print(f"El mejor modelo (basado en CV RMSE) es Ridge Polinómico con RMSE = {best_rmse_cv_ridge:.4f}")
    best_model = grid_search_ridge.best_estimator_
    best_model_name = "Ridge Polinómico"
    best_params = grid_search_ridge.best_params_
    best_cv_rmse = best_rmse_cv_ridge

print(f"\nModelo seleccionado final: {best_model_name}")
print(f"Mejores Hiperparámetros encontrados:")
print(f"  Grado Polinómico: {best_params['poly__degree']}")
print(f"  Alpha de Regularización: {best_params['regressor__alpha']}")
print(f"RMSE estimado (Validación Cruzada): {best_cv_rmse:.4f}")


print("\n--- Evaluando el MEJOR modelo en el CONJUNTO DE PRUEBA ---")

# Predecir en el conjunto de prueba usando el mejor pipeline encontrado
y_pred_test = best_model.predict(X_test)

# Calcular métricas en el conjunto de prueba
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)

print(f"\nResultados en el Conjunto de Prueba:")
print(f"  RMSE: {test_rmse:.4f}")
print(f"  R²: {test_r2:.4f}")

# (Opcional) Comparar con RMSE base (predecir siempre la media)
# base_rmse = np.sqrt(mean_squared_error(y_test, [y_train.mean()] * len(y_test)))
# print(f"  RMSE Baseline (predecir media): {base_rmse:.4f}")

print(f"\nRango de 'track_popularity' (Target): {y.min()} - {y.max()}")
print(f"Desviación estándar de 'track_popularity' en Test Set: {y_test.std():.4f}")


print("\n--- Proceso completado ---")